# ID wrangling
### In this notebook, we will match players from our Premier League data with players from our FIFA data using various techniques, and correct remaining outliers by hand.

In [1]:
import pandas as pd
from unidecode import unidecode
import numpy as np

In [169]:
fifa = pd.read_pickle("all_games.pkl")
pl = pd.read_pickle("all_seasons.pkl")

In [170]:
# All unique IDs in the FIFA data set.
fifa_unique = fifa.drop_duplicates(subset="id", keep='first')

In [171]:
# We find all unique players in the PL
pl_players = []

# List of all players in all matches
for row in pl['players']:
    for player in row:
        pl_players.append(player)

# Remove duplicates by making dict with player ids as keys
pl_players = list(dict((x[2], x) for x in pl_players).values())

# Transform to dataframe and find name duplicates, we will check these manually at the end.
pl_players = pd.DataFrame(pl_players, columns = ['name', 'position', 'id'])
pl_players[pl_players.duplicated(['name'], keep=False)]

,name,position,id
5,Aaron Ramsey,Midfielder,3548
288,Danny Ward,Forward,3831
381,Danilo,Defender,5328
857,Danny Ward,Goalkeeper,4522
932,Danilo,Midfielder,112510
1068,Aaron Ramsey,Midfielder,48285


In [172]:
# Rename both Danilos in the PL dataframe
pl_players.loc[pl_players['id'] == '5328', 'name'] = 'Danilo Luiz da Silva'
pl_players.loc[pl_players['id'] == '112510', 'name'] = 'Danilo Santos de Oliveira'

In [173]:
# Remove accents from letters in players names with unidecode, so they will be easier to merge
pl_players.loc[:,'name'] = pl_players['name'].apply(lambda x: unidecode(x))
fifa_unique.loc[:,'name'] = fifa_unique['name'].apply(lambda x: unidecode(x))
fifa_unique.loc[:,'short_name'] = fifa_unique['short_name'].apply(lambda x: unidecode(x))

In [174]:
# Join PL and FIFA dataframes on name. We join on pl_players because we need to merge only the players in our match data.
df = pd.merge(pl_players, fifa_unique, how='left', on='name')

In [175]:
# Check how many rows did not merge successfully
df['rating'].isna().sum() # 200 / 1136 rows

# Create df_leftovers for players that did not merge successfully
df_leftovers = df.loc[df['rating'].isnull()].iloc[:,0:3]
# Looking at the failed merges manually, we can see many names are more complicated in the FIFA dataframe, and PL uses simplified names. Also FIFA includes more special characters, PL tends to omit special characters and replace them by a standard letter.

In [176]:
# Merge leftovers on short names
df_2 = pd.merge(df_leftovers, fifa_unique, how='left', left_on='name', right_on='short_name')

# Check how many rows did not merge successfully
df_2['rating'].isna().sum() # 64 / 200 rows

# Create df_leftovers_2 for players that did not merge successfully
df_leftovers_2 = df_2.loc[df_2['rating'].isnull()].iloc[:,0:3]

In [177]:
# Create short names (first letter of first name + last name) for PL players to merge with short_name from FIFA
def shorten_name(name):
    first_name = name.split()[0]
    last_name = name.split()[-1]
    initial = first_name[0] + "."
    short_name = initial + " " + last_name
    return short_name

# Apply function to leftover pl_players
df_leftovers_2['short_name'] = df_leftovers_2['name_x'].apply(shorten_name)

In [178]:
# Merge leftovers on short names
df_3 = pd.merge(df_leftovers_2, fifa_unique, how='left', on='short_name')

# Check how many rows did not merge successfully
df_3['rating'].isna().sum() # 37 / 64 rows

# Create df_leftovers_3 for players that did not merge successfully
df_leftovers_3 = df_3.loc[df_3['rating'].isnull()].iloc[:,0:3]

In [179]:
# Add blank fifa_unique columns to df_leftovers_3
for col in fifa_unique.columns:
    if col not in df_leftovers_3.columns:
        df_leftovers_3[col] = np.nan

# Check if PL name is contained in FIFA name, if so, copy the values of the FIFA rows and add paste them in the corresponding PL rows.
for pl_index, pl_row in df_leftovers_3.iterrows():
    pl_name = pl_row['name_x']
    for _, fifa_row in fifa_unique.iterrows():
        fifa_name = fifa_row['name']
        if pl_name in fifa_name:
            df_leftovers_3.loc[pl_index, fifa_unique.columns] = fifa_row
            break

# Set df_4 to the new df_leftovers_3
df_4 = df_leftovers_3
# Check how many rows did not merge successfully
df_4['rating'].isna().sum() # 14 / 37 rows

# Create df_leftovers_4 for players that did not merge successfully
df_leftovers_4 = df_4.loc[df_4['rating'].isnull()].iloc[:,0:3]

C:\Users\victo\AppData\Local\Temp\ipykernel_12100\2650919180.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fabricio Agosto Ramirez' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_leftovers_3.loc[pl_index, fifa_unique.columns] = fifa_row
C:\Users\victo\AppData\Local\Temp\ipykernel_12100\2650919180.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fabricio' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_leftovers_3.loc[pl_index, fifa_unique.columns] = fifa_row
C:\Users\victo\AppData\Local\Temp\ipykernel_12100\2650919180.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fulham' has dtype incompatible with float64, please explicitly cast to

In [180]:
# Since we now have very few remaining players (14), that are difficult to merge with the same methods, we will merge them manually (nicknames, typos, unidecode didn't work, ...)
# Add blank fifa_unique columns to df_leftovers_4
for col in fifa_unique.columns:
    if col not in df_leftovers_4.columns:
        df_leftovers_4[col] = np.nan

# Manually copy and paste row data
df_leftovers_4.loc[4, fifa_unique.columns] = fifa_unique.loc[184256]
df_leftovers_4.loc[8, fifa_unique.columns] = fifa_unique.loc[22767]
df_leftovers_4.loc[9, fifa_unique.columns] = fifa_unique.loc[366]
df_leftovers_4.loc[13, fifa_unique.columns] = fifa_unique.loc[252]
df_leftovers_4.loc[18, fifa_unique.columns] = fifa_unique.loc[142101]
df_leftovers_4.loc[20, fifa_unique.columns] = fifa_unique.loc[142175]
df_leftovers_4.loc[26, fifa_unique.columns] = fifa_unique.loc[184431]
df_leftovers_4.loc[28, fifa_unique.columns] = fifa_unique.loc[101070]
df_leftovers_4.loc[35, fifa_unique.columns] = fifa_unique.loc[34]
df_leftovers_4.loc[38, fifa_unique.columns] = fifa_unique.loc[101058]
df_leftovers_4.loc[45, fifa_unique.columns] = fifa_unique.loc[251]
df_leftovers_4.loc[46, fifa_unique.columns] = fifa_unique.loc[55809]
df_leftovers_4.loc[50, fifa_unique.columns] = fifa_unique.loc[58]
df_leftovers_4.loc[51, fifa_unique.columns] = fifa_unique.loc[23158]

# Set df_5 to df_leftovers_4
df_5 = df_leftovers_4

C:\Users\victo\AppData\Local\Temp\ipykernel_12100\3322496898.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Alvaro Borja Morata Martin' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_leftovers_4.loc[4, fifa_unique.columns] = fifa_unique.loc[184256]
C:\Users\victo\AppData\Local\Temp\ipykernel_12100\3322496898.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Morata' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_leftovers_4.loc[4, fifa_unique.columns] = fifa_unique.loc[184256]
C:\Users\victo\AppData\Local\Temp\ipykernel_12100\3322496898.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Chelsea' has dtype incompatible with float64, please exp

In [181]:
# Make a dictionary with PL ids as keys and FIFA ids as values. Do this for our 5 dataframes.
dict_1 = {}
for _, row in df[~df['rating'].isna()].iterrows():
    dict_1[row['id_x']] = row['id_y']

dict_2 = {}
for _, row in df_2[~df_2['rating'].isna()].iterrows():
    dict_2[row['id_x']] = row['id']

dict_3 = {}
for _, row in df_3[~df_3['rating'].isna()].iterrows():
    dict_3[row['id_x']] = row['id']

dict_4 = {}
for _, row in df_4[~df_4['rating'].isna()].iterrows():
    dict_4[row['id_x']] = row['id']

dict_5 = {}
for _, row in df_5.iterrows():
    dict_5[row['id_x']] = row['id']

# Extend dict_1 with the contents of other dicts.
dict_1.update(dict_2)
dict_1.update(dict_3)
dict_1.update(dict_4)
dict_1.update(dict_5)
id_dict = dict_1
len(id_dict) # 1124 / 1129

# There are some players missing so we find which ids are missing in the dict
for _, row in pl_players.iterrows():
    if row['id'] not in id_dict:
        print(row)

# We add these players' id key value pairs to the dict manually
dict_6 = {'20559':'212831', '32162':'227236', '4999':'200104', '67634':'255253', '43087':'234457'}
id_dict.update(dict_6)

# We now manually correct the ids of the players with duplicate names found earlier
pl_players[pl_players.duplicated(['name'], keep=False)]
id_dict['3831'] = '194319'
id_dict['4522'] = '207998'
id_dict['3548'] = '186561'
id_dict['48285'] = '266270'

In [1]:
# Save id_dict to pickle.
id_list = list(id_dict.items())
id_df = pd.DataFrame(id_list, columns = ['keys', 'values'])
id_df.to_pickle("id_df.pkl")

NameError: name 'id_dict' is not defined